In [1]:
from mistralai import Mistral
from dotenv import load_dotenv
from mistralai.models import UserMessage, SystemMessage

load_dotenv()

import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))
from src.board.connect_four_board import ConnectFourBoard
from src.types.move import Move

In [4]:
from typing import get_args
from src.types.model_provider_name import ModelProviderName
"mistral" in get_args(ModelProviderName)

True

In [6]:
Move(**{"col" : 3, "row": 5})

Move(col=3, row=5)

In [39]:
system_prompt = SystemMessage(
    content="""You are an expert Connect Four player.

GAME RULES:
- 7 columns (numbered 0-6), 6 rows
- Pieces drop to the lowest empty row in chosen column
- Win by connecting 4 pieces horizontally, vertically, or diagonally
- You are piece 1, opponent is piece 2

STRATEGY (check in this order):
1. If YOU can win this turn, play that winning move
2. If OPPONENT can win next turn, block them
3. Otherwise, play strategically (center columns are strongest)

OUTPUT FORMAT:
Return ONLY valid JSON: {"col": integer, "row": integer}
No explanations. No other text.

EXAMPLES:
Example 1 - Take the win:
Board: Row5: 1 1 1 0 0 0 0
Analysis: You can win by playing column 3
Answer: {{"col": 3}}

Example 2 - Block opponent:
Board: Row5: 2 2 2 0 0 0 0
Analysis: Opponent wins if you don't block column 3
Answer: {{"col": 3, "row": 5}}

""")

In [12]:
from src.types.piece_enum import PieceEnum



def get_s(p: PieceEnum):

    s = f"""ArithmeticError
        {PieceEnum(3 - p)}
    """

    return s

get_s(PieceEnum.CPU)

'ArithmeticError\n        1\n    '

In [64]:
def create_prompt(board: ConnectFourBoard, piece) -> UserMessage:
    s = "CURRENT BOARD (0=empty, 1=YOU, 2=OPPONENT):\n"
    s += "Col : 0 1 2 3 4 5 6\n"
    # print col i
    for i, r in enumerate(board.state):
        s += f"Row{i}: " + " ".join(map(str, r)) + "\n"

    moves = "\n".join(str(m.model_dump()) for m in board.get_possible_moves())

    s += f"""
You are piece {piece}.
Opponent is piece {3 - piece}.

here are the possible moves you can make
{moves}

ANALYZE:
1. Can YOU win this turn? If yes, play that column.
2. Can OPPONENT win next turn? If yes, block that column.
3. Otherwise choose strategically.


Return ONLY: {{"col": integer, "row": integer}}
"""
    return UserMessage(content=s)

In [58]:
from src.types.move import Move

Move(col=3, row=5).model_dump()

{'col': 3, 'row': 5}

In [65]:
board = [
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 2],
            [0, 0, 0, 0, 0, 0, 2],
            [1, 1, 0, 0, 0, 0, 2],
]
board = ConnectFourBoard(initial_state=board)
piece = 1
prompt = create_prompt(board, piece)

In [66]:
print(prompt.content)

CURRENT BOARD (0=empty, 1=YOU, 2=OPPONENT):
Col : 0 1 2 3 4 5 6
Row0: 0 0 0 0 0 0 0
Row1: 0 0 0 0 0 0 0
Row2: 0 0 0 0 0 0 0
Row3: 0 0 0 0 0 0 2
Row4: 0 0 0 0 0 0 2
Row5: 1 1 0 0 0 0 2

You are piece 1.
Opponent is piece 2.

here are the possible moves you can make
{'col': 0, 'row': 4}
{'col': 1, 'row': 4}
{'col': 2, 'row': 5}
{'col': 3, 'row': 5}
{'col': 4, 'row': 5}
{'col': 5, 'row': 5}
{'col': 6, 'row': 2}

ANALYZE:
1. Can YOU win this turn? If yes, play that column.
2. Can OPPONENT win next turn? If yes, block that column.
3. Otherwise choose strategically.


Return ONLY: {"col": integer, "row": integer}



In [31]:
with Mistral(
    api_key=os.environ["MISTRAL_API_KEY"],
) as mistral:
    models = mistral.models.list().data
    print("Available models:", "\n".join(m.name for m in models))

Available models: mistral-medium-2505
mistral-medium-2508
mistral-medium-2508
mistral-medium-2508
open-mistral-nemo
open-mistral-nemo
open-mistral-nemo
open-mistral-nemo
mistral-large-2411
pixtral-large-2411
pixtral-large-2411
pixtral-large-2411
codestral-2508
codestral-2508
devstral-small-2507
devstral-medium-2507
devstral-2512
devstral-2512
devstral-2512
devstral-2512
labs-devstral-small-2512
labs-devstral-small-2512
mistral-small-2506
mistral-small-2506
labs-mistral-small-creative
magistral-medium-2509
magistral-medium-2509
magistral-small-2509
magistral-small-2509
voxtral-mini-2507
voxtral-mini-2507
voxtral-small-2507
voxtral-small-2507
mistral-large-2512
mistral-large-2512
ministral-3b-2512
ministral-3b-2512
ministral-8b-2512
ministral-8b-2512
ministral-14b-2512
ministral-14b-2512
open-mistral-7b
open-mistral-7b
open-mistral-7b
pixtral-12b-2409
pixtral-12b-2409
pixtral-12b-2409
ministral-3b-2410
ministral-8b-2410
codestral-2501
codestral-2501
codestral-2501
mistral-small-2501
mist

In [2]:
with Mistral(
    api_key=os.environ["MISTRAL_API_KEY"],
) as mistral:
    response = mistral.chat.complete(
        model="devstral-2512",
        response_format={"type": "json_object"},
        messages=[system_prompt, prompt],
    )
    print("Mistral's Move Response:", response.choices[0].message.content)

NameError: name 'Mistral' is not defined

In [ ]:
import json
c = json.loads(str(f"{response.choices[0].message.content}"))


NameError: name 'response' is not defined

[Move(col=0, row=5),
 Move(col=1, row=5),
 Move(col=2, row=5),
 Move(col=3, row=5),
 Move(col=4, row=5),
 Move(col=5, row=5),
 Move(col=6, row=5)]

In [17]:
import inspect
from typing import get_type_hints

TOOLS = []

PYTHON_TO_JSON = {
    str: "string",
    int: "integer",
    float: "number",
    bool: "boolean",
}

def mistral_tool(description: str):
    def decorator(fn):
        sig = inspect.signature(fn)
        hints = get_type_hints(fn)

        properties = {
            name: {
                "type": PYTHON_TO_JSON.get(hints.get(name, str), "string")
            }
            for name in sig.parameters
        }

        tool_spec = {
            "type": "function",
            "function": {
                "name": fn.__name__,
                "description": description,
                "parameters": {
                    "type": "object",
                    "properties": properties,
                    "required": list(sig.parameters),
                },
            },
        }

        TOOLS.append(tool_spec)
        return fn
    return decorator


In [21]:
@mistral_tool(description="Get payment status of a transaction")
def retrieve_payment_status(transaction_id: str, date: str):
    pass

In [22]:
TOOLS

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string'},
     'date': {'type': 'string'}},
    'required': ['transaction_id', 'date']}}}]

In [ ]:
from pydantic import BaseModel
from typing import Literal, Optional


class PropertySchema(BaseModel):
    name: str
    type: Literal["string", "number", "integer", "boolean", "object", "array"]
    description: Optional[str] = None


class ParametersSchema(BaseModel):
    type: Literal["object"]
    properties: list[PropertySchema]
    required: Optional[list[str]] = None


class ToolSpecFunction(BaseModel):
    name: str
    description: str
    parameters: ParametersSchema


class ToolSpec(BaseModel):
    type: Literal["function"]
    function: ToolSpecFunction

In [ ]:
tool = ToolSpec(
    type="function",
    function=ToolSpecFunction(
        name="example_tool",
        description="An example tool",
        parameters=ParametersSchema(
            type="object",
            properties=[
                PropertySchema(name="param1", type="string", description="The first parameter"),
                PropertySchema(name="param2", type="integer", description="The second parameter"),
            ],
            required=["param1", "param2"],
        ),
    )
)